# 우선 중구만 봄!

In [1]:
import pandas as pd
import numpy as np
import math

import time
import os

# from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

import folium
from geopy.geocoders import Nominatim

import networkx as nx



데이터 시점은 최대한 2021.12월 기준 자료 사용(단, 없을 경우 최대한 비슷한 시기의 자료 사용)
* node : 충전소, 교통량 관측소
    - attribute(충전소) : 급속, 완속 충전기
    - attribute(관측소) : 인구, 강우, 상권, 교통량 등<br>
    --- 추가 가능한 자료 : 행정동별 전기차 현황, 소득평균
* edge : 이동거리(임계치 설정)
    - weight : 걸리는 시간, 교통량(?)

** 검토해야하는 것 : 그리드에 수치 올리고 연결하는 것    
  

In [61]:
#node_traffic1 = pd.read_excel('./data/202112_traffic_v0.1.xlsx') # traffic
# 구, 동 정보있는것
node_traffic2 = pd.read_csv('./data/202209_seoul_traffic_point_v0.2.csv', index_col=0) # traffic 구, 동 정보
# 2만개 node_charging1 = pd.read_excel('./data/202210_elec_monitor.xlsx') # charging station
# 아래는 한전에서 관리하는것만 약 620개
node_charging2 = pd.read_csv('./data/electric charging station_address_v0.5.csv') # charging station
node_apt = pd.read_csv('./data/node_apartment_v0.2.csv', encoding = 'cp949') # apartment


In [62]:
node_apt_junggu = node_apt[node_apt['주소(시군구)'] == '중구'] # 37개
node_apt_junggu.reset_index(drop=True, inplace=True)
print(node_apt_junggu.head())

      번호    k-아파트코드          k-아파트명 k-단지분류(아파트,주상복합등등)            kapt도로명주소  \
0   2199  A10080701        LIG서울역리가                아파트  서울특별시 중구 만리재로33길 21   
1  18161  A10026881     서울역센트럴자이아파트                아파트    서울특별시 중구 만리재로 175   
2  18841  A10026517  서울역한라비발디센트럴아파트                아파트    서울특별시 중구 만리재로 177   
3  18221  A10026849           묵정아파트                아파트    서울특별시 중구 서애로1길 34   
4  15083  A10027364     덕수궁롯데캐슬아파트                 아파트                  NaN   

            kapt도로명주소2 주소(시도)k-apt주소split 주소(시군구) 주소(읍면동)               나머지주소  \
0  서울특별시 중구 만리재로33길 21                 서울      중구   만리동1가            LIG서울역리가   
1    서울특별시 중구 만리재로 175                 서울      중구   만리동2가    서울특별시 중구 만리재로175   
2    서울특별시 중구 만리재로 177                 서울      중구   만리동2가   서울역한라비발디아파트 관리사무소   
3    서울특별시 중구 서애로1길 34                 서울      중구     묵정동                 NaN   
4          서울 중구 순화동                   서울      중구     순화동  서울특별시 중구 서소문로9길 28   

   ... k-전용면적별세대현황(60㎡이하) k-전용면적별세대현황(

In [4]:
node_charging2.head() # AC는 완속충전, DC는 급속충전

,NO,location,address,gu,dong,latitude,longitude,dc,ac,cars
0,0,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,강남구,자곡동,37.470507,127.089821,1,4,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
1,1,LH서울지사,서울특별시 강남구 선릉로 121길 12,강남구,선릉로,0.000000,0.000000,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
2,2,가람아파트,서울특별시 강남구 일원동 일원로 127,강남구,일원동,37.483147,127.085369,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
3,3,강남구청 공영주차장,서울특별시 강남구 삼성동 16-1,강남구,삼성동,37.514784,127.054362,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
4,4,강남데시앙파크,서울특별시 강남구 세곡동 헌릉로590길 63,강남구,세곡동,37.463011,127.103861,1,1,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."


In [5]:
node_charging2_junggu = node_charging2[node_charging2['gu'] == '중구'] # 우선 중구만 테스트
node_traffic2_junggu = node_traffic2[node_traffic2['구'] == '중구'] # 우선 중구만 테스트
print(node_charging2_junggu.head())
print(node_traffic2_junggu.head())

      NO        location                 address  gu    dong   latitude  \
584  584     LIG서울역리가아파트      서울특별시 중구 만리동1가 231  중구  만리동1가    0.000000   
585  585          국립중앙극장  서울특별시 중구 장충동2가 산 14-67  중구  장충동2가    0.000000   
586  586  남산롯데캐슬아이리스 아파트         서울특별시 중구 소공로 35  중구    소공로   37.559763   
587  587         남산센트럴자이  서울특별시 중구 충무로4가 퇴계로 235  중구  충무로4가   37.561051   
588  588          남산케이블카   서울특별시 중구 회현동1가 산 1-19  중구  회현동1가    0.000000   

      longitude  dc  ac                                               cars  
584    0.000000   0   2  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
585    0.000000   3   0                                        SM3 Z.E,테슬라  
586  126.982249   1   0  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
587  126.991072   0   2  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
588    0.000000   1   0  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
     구       동  지점번호         지점명칭         위도          경도  N_University  \
8   중구    흥인동

In [6]:
node_charging2_junggu.reset_index(drop=True, inplace=True)
node_traffic2_junggu.reset_index(drop=True, inplace=True)
print(node_charging2_junggu.head())
print(node_traffic2_junggu.head())

    NO        location                 address  gu    dong   latitude  \
0  584     LIG서울역리가아파트      서울특별시 중구 만리동1가 231  중구  만리동1가    0.000000   
1  585          국립중앙극장  서울특별시 중구 장충동2가 산 14-67  중구  장충동2가    0.000000   
2  586  남산롯데캐슬아이리스 아파트         서울특별시 중구 소공로 35  중구    소공로   37.559763   
3  587         남산센트럴자이  서울특별시 중구 충무로4가 퇴계로 235  중구  충무로4가   37.561051   
4  588          남산케이블카   서울특별시 중구 회현동1가 산 1-19  중구  회현동1가    0.000000   

    longitude  dc  ac                                               cars  
0    0.000000   0   2  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
1    0.000000   3   0                                        SM3 Z.E,테슬라  
2  126.982249   1   0  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
3  126.991072   0   2  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
4    0.000000   1   0  SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...  
    구       동  지점번호         지점명칭         위도          경도  N_University  \
0  중구    흥인동   A-09     퇴계로(신당역)  37.5

### 1. 노드생성 : 충전소(nodes_charging1), 교통량 측정지점(nodes_traffic)
  1-1. 충전소 [위도, 경도] 기준으로 노드 표시(총 29,903개 -> 600여개 한전것만 사용, 중구기준 총 19개)<br>
    - attribute : 충전기 타입(급속 db, 완속 ac), 위도, 경도<br>    
  1-2. 교통량 측정지점 [위도 , 경도] 기준으로 노드 표시(총 136개, 중구기준 총 14개)<br>
    - attribute : 강우량, 대학교, 위도, 경도, <br>
  1-3. 아파트 [위도, 경도] 기준으로 노드 표시(중구기준 총 37개)<br>
    - attribute : 세대수, 위도, 경도

In [7]:
node_charging2_junggu.head()

,NO,location,address,gu,dong,latitude,longitude,dc,ac,cars
0,584,LIG서울역리가아파트,서울특별시 중구 만리동1가 231,중구,만리동1가,0.000000,0.000000,0,2,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
1,585,국립중앙극장,서울특별시 중구 장충동2가 산 14-67,중구,장충동2가,0.000000,0.000000,3,0,"SM3 Z.E,테슬라"
2,586,남산롯데캐슬아이리스 아파트,서울특별시 중구 소공로 35,중구,소공로,37.559763,126.982249,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
3,587,남산센트럴자이,서울특별시 중구 충무로4가 퇴계로 235,중구,충무로4가,37.561051,126.991072,0,2,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."
4,588,남산케이블카,서울특별시 중구 회현동1가 산 1-19,중구,회현동1가,0.000000,0.000000,1,0,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E..."


######## 아래 코드부터 해결해야하는 geocoding API 호출 #################################

In [9]:
#Geocoding NAVER
import urllib
from urllib import parse
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

import json

# NCP 콘솔에서 복사한 클s라이언트ID와 클라이언트Secret 값
client_id = '4tzcaqi2vc'
client_secret = 'N2QjrXLyFDyLomnTPH9Lavcq5W48Php2dLfizRxk'

### for이용
temp = [] # 
addresses = [] # 위도, 경도 변환을 위해

for i in range(0, len(node_charging2_junggu)) : ### 
    print(f'i = {i}')
    temp.append([])

    for cell in node_charging2_junggu.iloc[i, :] : 
        print(f'test2')

        #print(f'cell = {cell}')
        temp[-1].append(cell)
        print(temp)
    


i = 0
test2
[[584]]
test2
[[584, 'LIG서울역리가아파트']]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231']]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구']]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ']]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0]]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0]]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0, 0]]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0, 0, 2]]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0, 0, 2, 'SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트EV,테슬라']]
i = 1
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0, 0, 2, 'SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트EV,테슬라'], [585]]
test2
[[584, 'LIG서울역리가아파트', '서울특별시 중구 만리동1가 231', '중구', '만리동1가 ', 0.0, 0.0, 0, 2, 'SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트EV,테슬라'], [585, '국립중앙극장']]
test2
[[584, 'LIG서울역

In [12]:
### 주소1개만 호출시도 ###

encoding_address = urllib.parse.quote_plus('서울특별시 중구 다동 85-4')

url = f'http://naveropenapi.apigw.ntruss.com/map-geocode/v2/goecode?query={encoding_address}'
# headers = {'User-Agent':'Chrome/66.0.3359.181'}
request = urllib.request.Request(url) # , headers = headers)
request.add_header('X-NCP-APIGW-API-KEY-ID', client_id) 
request.add_header('X-NCP-APIGW-API-KEY', client_secret)


In [ ]:
'''
for add in range(0, 1) : # data['도로명주소']:
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.'http://naveropenapi.apigw.ntruss.com/map-geocode/v2/goecode?query={http://naveropenapi.apigw.ntruss.com/map-geocode/v2/goecode?add_header('X-NCP-APIGW-API-KEY-ID', F___)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] == [] :
                print("'result' not exist!")
                latitude = None
                longitude = None
            else:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
        else:
            print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([latitude, longitude])


np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"도로명": data['도로명주소'].values,
                              "위도": np_geo_coordi[:, 0],
                              "경도": np_geo_coordi[:, 1]})
'''

In [ ]:

response = urllib.request.urlopen(request)


In [ ]:
'''
response_code = response.getcode() # 불러온 사이트가 정상인지 확인

print(response_code)
'''

In [14]:
### 주소 for문으로 호출 (error)###
'''
for idx in range(0, len(node_charging2_junggu)) :
    addresses.append(temp[i][2]) # 주소 전체값

count = 0 

for address in addresses : 
    print(address)
    encoding_address = urllib.parse.quote_plus(address)
    # url을 읽을 수 있게 ascii값으로 변환을 위해 urllib.parse.quote_plus 사용
    url = f'http://naveropenapi.apigw.ntruss.com/map-geocode/v2/goecode?query={encoding_address}'
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id) 
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode() # 불러온 사이트가 정상인지 확인


    if response_code == 200 : # 정상
        try : 
            response_body = response.read()
            # return된 JSON파일을 파싱해서 사전데이터로 만들어주는 json 라이브러리 사용

            data = json.loades(response_body)
            # node_charging2_junggu.cell(row=count, coloumn=11).value = data['addresses'][0]['y']
            node_charging2_junggu.iloc[count, 11] = data['addresses'][0]['y']
            # node_charging2_junggu.cell(row=count, coloumn=12).value = data['addresses'][0]['x']
            node_charging2_junggu.iloc[count, 12] = data['addresses'][0]['x']
            print(node_charging2_junggu.iloc[count, 3]) # 경도위도 변환을 완료한 주소 출력
            count += 1 # 행 아래로 한 줄 내리기
        except : 
            #node_charging2_junggu.cell(row=count, coloumn=11).value = 'error'
            #node_charging2_junggu.cell(row=count, coloumn=12).value = 'error'
            node_charging2_junggu.iloc[count, 11] = 'error'
            node_charging2_junggu.iloc[count, 12] = 'error'

            count += 1
            print(f'error : {count}')

node_charging2_junggu.head()
'''

서울특별시 중구 다동 85-4


URLError: <urlopen error [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다>

######## 위 코드까지 해결해야하는 API 호출 #################################

direction 5 호출 driving api 호출 코드 start

https://api.ncloud-docs.com/docs/ai-naver-mapsdirections-driving

In [ ]:
### 중간 경유를 5개까지 추가할 수 있는 Directions 5

option = ''
# option : 탐색옵션 최대 3개(빠르기 정도 선택), 기본옵션 traoptimal  

def get_optimal_route(start, goal, waypoints=['',''], option=option ) :
    ### client_id = '발급받은 client id' # 위에서 선언완료
    ### client_secret = '발급받은 client secret' # 위에서 선언완료
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving? \
    start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}\
    &waypoint={waypoint[0]},{waypoint[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
    
    if (res == 200) :
        response_body = response.read().decode('utf-8')
        return json.loads(response_body)
            
    else :
        print('ERROR')
        
get_optimal_route(start, goal, option=option)

In [ ]:
# 목적지까지의 정보(총 거리, 총 소요시간, 요금 등)
message = {
    'message' : results['message'], # traoptimal은 option 에 따라 다르게 줄 것
    'option' : list(results['route'].keys())[0],
    'total_distance' : results['route']['traoptimal'][0]['summary']['distance'],
    'total_duration' : results['route']['traoptimal'][0]['summary']['duration'],
    'fares' : {'toll' : results['route']['traoptimal'][0]['summary']['tollFare'],
              'taxi' : results['route']['traoptimal'][0]['summary']['taxiFare'],
              'fuel' : results['route']['traoptimal'][0]['summary']['fuelPrice']}
}
print(message)

# *-- 목적지까지의 guidence와 각각의 거리, 소요시간 정보 추출 --*
temp = [ (guide['instructions'], guide['distance'] , guide['duration'] / 1000 ) \
for guide in results['route']['traoptimal'][0]['guide'] ]

print(temp)

############ direction 5 호출 driving api 호출 코드 end ############

In [18]:
'''
for idx in node_charging1_junggu['주소'] : 
    ### print(idx)

    result = geocoding(idx)

    ### sta_data2['위도'] = test1[0]
    ### sta_data2['경도'].append(test1[1])
    ### sta_data1['위도'], sta_data1['경도'] = test1
    #latitude, longitude = test1
    latitude.append(result[0]) 
    ### latitude.append(geocoding(i)[0]) # x_y = [geo.latitude, geo.longitude]
    ###print(f'latitude = {latitude}')

    longitude.append(result[1]) # x_y = [geo.latitude, geo.longitude]
    ### longitude.append(geocoding(i)[1]) # x_y = [geo.latitude, geo.longitude]
    ### print(f'latitude = {latitude} , latitude = {latitude}')
    ### print('\n')

test = pd.DataFrame({'add': node_charging1['주소'], 'lat':latitude,'long':longitude})

test
'''

"\nfor idx in node_charging1_junggu['주소'] : \n    ### print(idx)\n\n    result = geocoding(idx)\n\n    ### sta_data2['위도'] = test1[0]\n    ### sta_data2['경도'].append(test1[1])\n    ### sta_data1['위도'], sta_data1['경도'] = test1\n    #latitude, longitude = test1\n    latitude.append(result[0]) \n    ### latitude.append(geocoding(i)[0]) # x_y = [geo.latitude, geo.longitude]\n    ###print(f'latitude = {latitude}')\n\n    longitude.append(result[1]) # x_y = [geo.latitude, geo.longitude]\n    ### longitude.append(geocoding(i)[1]) # x_y = [geo.latitude, geo.longitude]\n    ### print(f'latitude = {latitude} , latitude = {latitude}')\n    ### print('\n')\n\ntest = pd.DataFrame({'add': node_charging1['주소'], 'lat':latitude,'long':longitude})\n\ntest\n"

In [ ]:
'''
####### 도로명주소 위도 경도 값으로 바꿔주는 함수(현재 이걸로 사용 => null값 return하는 값이 있음-> api써야할듯) ########
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        ### print(f'geo_local.geocode(address) = {geo_local.geocode(address)}')
 
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0, 0]

#####주소를 위,경도 값으로 변환하기 #####
latitude = [] # 위도
longitude =[] # 경도
result = pd.DataFrame(columns = ['lat', 'long'])
'''

In [21]:
node_charging2_junggu.info() # 19개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NO         19 non-null     int64  
 1   location   19 non-null     object 
 2   address    19 non-null     object 
 3   gu         19 non-null     object 
 4   dong       19 non-null     object 
 5   latitude   19 non-null     float64
 6   longitude  19 non-null     float64
 7   dc         19 non-null     int64  
 8   ac         19 non-null     int64  
 9   cars       19 non-null     object 
dtypes: float64(2), int64(3), object(5)
memory usage: 1.6+ KB


노드 생성하는 곳

In [63]:
G = nx.Graph()
G.number_of_nodes()

0

In [66]:
# 한전운영 중구 충전소 : 19개 nodes

for idx, row in node_charging2_junggu.iterrows(): 
    # add node to Graph G
    G.add_nodes_from([
          (row['location'], 
          {'dong':row['dong'],
          'latitude':row['latitude'], 
          'longitude':row['longitude'], 
          'dc':row['dc'], 
          'ac':row['ac'], 
          'node_type':'charging'}) ])
G.number_of_nodes() 

19

In [76]:
### G.nodes['국립중앙극장']

{'dong': '장충동2가 ',
 'latitude': 0.0,
 'longitude': 0.0,
 'dc': 3,
 'ac': 0,
 'node_type': 'charging'}

In [77]:
node_traffic2_junggu.columns   

Index(['구', '동', '지점번호', '지점명칭', '위도', '경도', 'N_University', 'N_Commercial',
       'N_Electric_car', 'Rain'],
      dtype='object')

In [80]:
# 교통량 측정지점 중구 총 14개 nodes
for idx, row in node_traffic2_junggu.iterrows() :
  G.add_nodes_from([
      (row['지점명칭'], 
      {'dong':row['동'], 
      '위도':row['위도'], 
      '경도':row['경도'],
      'N_University':row['N_University'],
      'N_Commercial':row['N_Commercial'],
      'N_Electric_car':row['N_Electric_car'],
      'Rain':row['Rain'], 
      'node_type':'traffic'})])     
G.number_of_nodes() # 19 + 14 = 33

33

In [83]:
G.nodes['남산1호터널']

{'dong': '예장동 ',
 '위도': 37.542406,
 '경도': 127.001356,
 'N_University': 0.0,
 'N_Commercial': 1256.0,
 'N_Electric_car': 61158.0,
 'Rain': 0.0004847309743092,
 'node_type': 'traffic'}

In [84]:
node_apt_junggu.columns

Index(['번호', 'k-아파트코드', 'k-아파트명', 'k-단지분류(아파트,주상복합등등)', 'kapt도로명주소',
       'kapt도로명주소2', '주소(시도)k-apt주소split', '주소(시군구)', '주소(읍면동)', '나머지주소',
       '주소(도로명)', '주소(도로상세주소)', 'k-세대타입(분양형태)', 'k-관리방식', 'k-전체동수', 'k-전체세대수',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '세대전기계약방법',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y'],
      dtype='object')

In [87]:
### 아파트 노드 중구 전체 총 37개
for idx, row in node_apt_junggu.iterrows() :
  G.add_nodes_from([
      (row['k-아파트명'], 
      {'dong':row['주소(읍면동)'],
      'latitude':row['좌표X'], 
      'longitude':row['좌표Y'],
      'household':row['k-전체세대수'], 
      'parking_space':row['주차대수'],
      'node_type':'apt'})])     
G.number_of_nodes() ### 19(충전소) + 14(관측지점) + 37(아파트) = 70 2개,,?

68

In [88]:
G.nodes

NodeView(('LIG서울역리가아파트', '국립중앙극장', '남산롯데캐슬아이리스 아파트', '남산센트럴자이', '남산케이블카', '남산타운 아파트', '덕수궁 롯데캐슬', '동대문디자인플라자', '롯데캐슬베네치아', '묵정공원 공영주차장', '서울시청', '서울역(한화역사)', '서울전력관리처', '서울직할', '신세계 본점', '쌍용남산플래티넘 아파트', '약수동공영주차장', '전국은행연합회', '한국관광공사 옆 노상 공영주차장', '퇴계로(신당역)', '동호로(장충체육관)', '장충단로(장충단공원)', '퇴계로(회현역)', '세종대로(서울역)', '서소문로(시청역)', '세종대로(시청역2)', '을지로(을지로3가역)', '칠패로(숭례문)', '남산1호터널', '남산3호터널', '소월로(회현역)', '소파로(숭의여자대학교)', '동작대교', 'LIG서울역리가', '서울역센트럴자이아파트', '서울역한라비발디센트럴아파트', '묵정아파트', '덕수궁롯데캐슬아파트 ', '순화동포스코더샵', '한진해모로', '신당삼성임대', '신당푸르지오', '약수하이츠아파트(임대)', '신당현대', '신당삼성(분양)', '신당약수하이츠', '신당남산타운임대', '신당남산타운(분양)', '래미안신당하이베르', '청구e편한세상(분양)', '청구e편한세상임대', '래미안신당하이베르임대', '파라다이스', '신당KCC스위첸아파트', '인현신성', '삼정아트테라스정동', '중림삼성래미안아파트', '중림삼성사이버빌리지', '충무로진양', '충무로엘크루메트로시티2', 'SH황학롯데캐슬베네치아', '황학아크로타워', '황학코아루', '남산SK리더스뷰', '남산롯데캐슬아이리스', '쌍용남산플래티넘', '청계천두산위브더제니스', '동대문와이즈캐슬'))

### 2. edge 만들기(충전소-교통관측지점) : 우선, 일괄 계산해서 edge만들어보기로 함.(되면, api로 변경)

In [90]:
# R is the Earth's radius
def distanceInKilometerByHaversine(x1, y1, x2, y2) : # 좌표1(위도, 경도), 좌표2(위도, 경도)
    distance = 0.0
    radius = 6371 # 지구 반지름(km)
    toRadian = math.pi / 180

    deltaLatitude = abs(x1 - x2) * toRadian
    deltaLongitude = abs(y1 - y2) * toRadian

    sinDeltaLat = math.sin(deltaLatitude/2)
    sinDeltaLng = math.sin(deltaLongitude/2)
    squareRoot = math.sqrt(
        sinDeltaLat * sinDeltaLat +
        math.cos(x1 * toRadian) * math.cos(x2 * toRadian) * sinDeltaLng * sinDeltaLng)

    distance = 2 * radius * math.asin(squareRoot)

    return distance # return 단위 : km

In [91]:
#test
#distanceInKilometerByHaversine(row['위도'], r['latitude'], row['경도'], r['longitude'])
distanceInKilometerByHaversine(37.547889, 126.997128, 35.158874, 129.043846) # return km
### 서울-부산 좌표 => distance = 322km 나와야함


322.72224899963976

In [29]:
'''edge 거리계산 됨(km)
for idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
    ## Calculate the distance between 충전소 and 교통량 관측지점
        d = distanceInKilometerByHaversine(row['위도'], row['경도'], r['latitude'], r['longitude'])
        
        print(f"교통량 관측지점 {row['지점명칭']} 위/경도 : {row['위도']}, {row['경도']}")
        print(f"충전소 {r['location']} 위/경도 : {r['latitude']}, {r['longitude']}")
        print(f"둘 사이의 거리 : {d}")
    
    # Link attribute : 'Source', 'Target' and weight = 'Length between them'
        # G.add_edge(row['Source'], row['Target'], weight = d)

        # chk 현재, 중구 안에서의 충전소-교통관측지점 까지의 거리를 weight으로 두었지만,
        # 추후, distance인 d <= 450m이하 거리면 weight을 1로 주고, 
        # distance가인 450m < d <= 1,000m 이면 weight을 0.5로 주는 걸로 수정(1km 이상될때는 안갈것 같음)
        G.add_edge(row['지점명칭'], r['location'], weight = d) 
'''

'edge 거리계산 됨(km)\nfor idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)\n    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)\n    ## Calculate the distance between 충전소 and 교통량 관측지점\n        d = distanceInKilometerByHaversine(row[\'위도\'], row[\'경도\'], r[\'latitude\'], r[\'longitude\'])\n        \n        print(f"교통량 관측지점 {row[\'지점명칭\']} 위/경도 : {row[\'위도\']}, {row[\'경도\']}")\n        print(f"충전소 {r[\'location\']} 위/경도 : {r[\'latitude\']}, {r[\'longitude\']}")\n        print(f"둘 사이의 거리 : {d}")\n    \n    # Link attribute : \'Source\', \'Target\' and weight = \'Length between them\'\n        # G.add_edge(row[\'Source\'], row[\'Target\'], weight = d)\n\n        # chk 현재, 중구 안에서의 충전소-교통관측지점 까지의 거리를 weight으로 두었지만,\n        # 추후, distance인 d <= 450m이하 거리면 weight을 1로 주고, \n        # distance가인 450m < d <= 1,000m 이면 weight을 0.5로 주는 걸로 수정(1km 이상될때는 안갈것 같음)\n        G.add_edge(row[\'지점명칭\'], r[\'location\'], weight = d) \n'

In [30]:
'''distance 길이에 따라 weight를 주려고 했었음
for idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
    ## Calculate the distance between 충전소 and 교통량 관측지점
        d = distanceInKilometerByHaversine(row['위도'], row['경도'], r['latitude'], r['longitude'])
        
        print(f"교통량 관측지점 {row['지점명칭']} 위/경도 : {row['위도']}, {row['경도']}")
        print(f"충전소 {r['location']} 위/경도 : {r['latitude']}, {r['longitude']}")
        print(f"둘 사이의 거리 : {d}")
    
    # Link attribute : 'Source', 'Target' and weight = 'Length between them'
        # G.add_edge(row['Source'], row['Target'], weight = d)

        # chk 현재, 중구 안에서의 충전소-교통관측지점 까지의 거리를 weight으로 두었지만,
        # 추후, distance인 d <= 450m이하 거리면 weight을 1로 주고, 
        # distance가인 450m < d <= 1,000m 이면 weight을 0.5로 주는 걸로 수정(1km 이상될때는 안갈것 같음)
        if d <= 450 :
            resul_d = 1.0
        elif (d <= 1000) and (d > 450) :
            result_d = 0.5
        else : 
            result_d = 0.0
            
        G.add_edge(row['지점명칭'], r['location'], weight = d) #result_d) 
'''

'distance 길이에 따라 weight를 주려고 했었음\nfor idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)\n    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)\n    ## Calculate the distance between 충전소 and 교통량 관측지점\n        d = distanceInKilometerByHaversine(row[\'위도\'], row[\'경도\'], r[\'latitude\'], r[\'longitude\'])\n        \n        print(f"교통량 관측지점 {row[\'지점명칭\']} 위/경도 : {row[\'위도\']}, {row[\'경도\']}")\n        print(f"충전소 {r[\'location\']} 위/경도 : {r[\'latitude\']}, {r[\'longitude\']}")\n        print(f"둘 사이의 거리 : {d}")\n    \n    # Link attribute : \'Source\', \'Target\' and weight = \'Length between them\'\n        # G.add_edge(row[\'Source\'], row[\'Target\'], weight = d)\n\n        # chk 현재, 중구 안에서의 충전소-교통관측지점 까지의 거리를 weight으로 두었지만,\n        # 추후, distance인 d <= 450m이하 거리면 weight을 1로 주고, \n        # distance가인 450m < d <= 1,000m 이면 weight을 0.5로 주는 걸로 수정(1km 이상될때는 안갈것 같음)\n        if d <= 450 :\n            resul_d = 1.0\n        elif (d <= 1000) and (d > 450) :\n       

교통량 관측지점 및 아파트 - 충전소 거리계산 for weight

In [112]:
# 거리에 따른 scale을 weight으로
scaler = MinMaxScaler()

d_traf = list()
for idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
    ## Calculate the distance between 충전소 and 교통량 관측지점
        d_traf.append(distanceInKilometerByHaversine(row['위도'], row['경도'], r['latitude'], r['longitude']))
        
        # print(f"교통량 관측지점 {row['지점명칭']} 위/경도 : {row['위도']}, {row['경도']}")
        # print(f"충전소 {r['location']} 위/경도 : {r['latitude']}, {r['longitude']}")
        # print(f"둘 사이의 거리 : {d}")
    
        # 거리에 scaler사용

d_apt = list()
for idx, row in node_apt_junggu.iterrows() : # 중구 아파트(37)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
    ## Calculate the distance between 충전소 and 교통량 관측지점
        d_apt.append(distanceInKilometerByHaversine(row['좌표X'], row['좌표Y'], r['latitude'], r['longitude']))
        
        # print(f"교통량 관측지점 {row['지점명칭']} 위/경도 : {row['위도']}, {row['경도']}")
        # print(f"충전소 {r['location']} 위/경도 : {r['latitude']}, {r['longitude']}")
        # print(f"둘 사이의 거리 : {d}")
    
        # 거리에 scaler사용


In [113]:
df_d_traf = pd.DataFrame(d_traf)
df_d_apt = pd.DataFrame(d_apt)
print(df_d_traf, df_d_apt)

scale_d_traf = scaler.fit_transform(df_d_traf)
scale_d_apt = scaler.fit_transform(df_d_apt)

print(scale_d_traf, scale_d_apt)
#        G.add_edge(row['지점명칭'], r['location'], weight = scale_d) #result_d) 

                0
0    13177.350793
1    13177.350793
2        3.471465
3        2.680757
4    13177.350793
..            ...
261  13176.805804
262      5.506283
263  13176.805804
264      6.190076
265      6.423563

[266 rows x 1 columns]                 0
0    13173.484512
1    13173.484512
2     6801.317585
3     6800.679609
4    13173.484512
..            ...
698  13176.910108
699   6804.169023
700  13176.910108
701   6804.060043
702   6803.666839

[703 rows x 1 columns]
[[1.00000000e+00]
 [1.00000000e+00]
 [2.51613589e-04]
 [1.91607817e-04]
 [1.00000000e+00]
 [1.00000000e+00]
 [3.31204424e-04]
 [3.76884960e-05]
 [3.33152939e-05]
 [7.67520503e-05]
 [3.71571777e-04]
 [1.00000000e+00]
 [2.98454634e-04]
 [2.77921540e-04]
 [1.00000000e+00]
 [2.44785121e-04]
 [1.00000000e+00]
 [2.81130280e-04]
 [2.57996029e-04]
 [9.99941252e-01]
 [9.99941252e-01]
 [1.54720738e-04]
 [9.74296800e-05]
 [9.99941252e-01]
 [9.99941252e-01]
 [2.41644883e-04]
 [7.92195899e-05]
 [1.35815553e-04]
 [1.84635466e-05

In [114]:
print(G.nodes['국립중앙극장']) # 충전소
print(G.nodes['퇴계로(신당역)']) # 관측지점
print(G.nodes['LIG서울역리가']) # 아파트

{'dong': '장충동2가 ', 'latitude': 0.0, 'longitude': 0.0, 'dc': 3, 'ac': 0, 'node_type': 'charging'}
{'dong': '흥인동 ', '위도': 37.565712, '경도': 127.020912, 'N_University': 0.0, 'N_Commercial': 1256.0, 'N_Electric_car': 61158.0, 'Rain': 0.0004847309743092, 'node_type': 'traffic'}
{'dong': '만리동1가', 'latitude': 126.9669575, 'longitude': 37.5558961, 'household': 181.0, 'parking_space': 221.0, 'node_type': 'apt'}


교통량-충전소 edges 생성(width = scaled_거리)

In [115]:
count = 0

for idx, row in node_traffic2_junggu.iterrows() : # 중구 교통량 관측지점(14)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
        # print(i, count, scale_d_traf[count])
        G.add_edges_from([
            (row['지점명칭'], r['location'], {'weight':float(scale_d_traf[count])})])
        count += 1

In [116]:
for edge in G.edges :
  print(edge)
  print(G.get_edge_data(edge[0], edge[1]))


('LIG서울역리가아파트', '퇴계로(신당역)')
{'weight': 1.0}
('LIG서울역리가아파트', '동호로(장충체육관)')
{'weight': 0.9999412521038565}
('LIG서울역리가아파트', '장충단로(장충단공원)')
{'weight': 0.9999322810841781}
('LIG서울역리가아파트', '퇴계로(회현역)')
{'weight': 0.9997591217111964}
('LIG서울역리가아파트', '세종대로(서울역)')
{'weight': 0.9997383826481505}
('LIG서울역리가아파트', '서소문로(시청역)')
{'weight': 0.9997172407977182}
('LIG서울역리가아파트', '세종대로(시청역2)')
{'weight': 0.9997280842498177}
('LIG서울역리가아파트', '을지로(을지로3가역)')
{'weight': 0.9998066102333456}
('LIG서울역리가아파트', '칠패로(숭례문)')
{'weight': 0.9997273913076404}
('LIG서울역리가아파트', '남산1호터널')
{'weight': 0.999963234420622}
('LIG서울역리가아파트', '남산3호터널')
{'weight': 0.9998767921262222}
('LIG서울역리가아파트', '소월로(회현역)')
{'weight': 0.9997597334764817}
('LIG서울역리가아파트', '소파로(숭의여자대학교)')
{'weight': 0.9998108199333667}
('LIG서울역리가아파트', '동작대교')
{'weight': 0.9999586415200256}
('국립중앙극장', '퇴계로(신당역)')
{'weight': 1.0}
('국립중앙극장', '동호로(장충체육관)')
{'weight': 0.9999412521038565}
('국립중앙극장', '장충단로(장충단공원)')
{'weight': 0.9999322810841781}
('국립중앙극장', '퇴계로(회현역)')
{'weigh

아파트-충전소 edge 생성

In [117]:
count = 0

for idx, row in node_apt_junggu.iterrows() : # 중구 아파트(37)
    for i, r in node_charging2_junggu.iterrows() : # 중구 충전소(19)
        # print(i, count, scale_d_apt[count])
        G.add_edges_from([
            (row['k-아파트명'], r['location'], {'weight':float(scale_d_apt[count])})])
        count += 1

In [118]:
G.number_of_nodes(), G.number_of_edges()

(68, 968)

In [ ]:
time = time.strftime('%H%M%S')
time
nx.write_gexf(G, 'test_' + time + '.gexf')

In [42]:
'''
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in node_traffic2_junggu.iterrows():
    start = tuple(merge_node2[['latitude','longitude']].iloc[0]) 
    end = tuple(merge_node2[['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm
'''

"\nkw = {'opacity': 0.5, 'weight': 2}\nfor ix, row in node_traffic2_junggu.iterrows():\n    start = tuple(merge_node2[['latitude','longitude']].iloc[0]) \n    end = tuple(merge_node2[['latitude','longitude']].iloc[0])\n    folium.PolyLine(\n        locations=[start, end],\n        color='blue',\n        line_cap='round',\n        **kw,\n    ).add_to(map_osm)\n# it takes some time.....\nmap_osm\n"

In [43]:
'''
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in node_traffic2_junggu.iterrows():
    start = tuple(merge_node2[['latitude','longitude']].iloc[0]) 
    end = tuple(merge_node2[['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm
'''

"\nkw = {'opacity': 0.5, 'weight': 2}\nfor ix, row in node_traffic2_junggu.iterrows():\n    start = tuple(merge_node2[['latitude','longitude']].iloc[0]) \n    end = tuple(merge_node2[['latitude','longitude']].iloc[0])\n    folium.PolyLine(\n        locations=[start, end],\n        color='blue',\n        line_cap='round',\n        **kw,\n    ).add_to(map_osm)\n# it takes some time.....\nmap_osm\n"

### 3. 지도에 node & edge 표시

In [120]:
std_point = tuple(node_charging2.head(1)[['latitude','longitude']].iloc[0])
std_point

(37.4705072, 127.0898211)

In [121]:
G.degree

DegreeView({'LIG서울역리가아파트': 51, '국립중앙극장': 51, '남산롯데캐슬아이리스 아파트': 51, '남산센트럴자이': 69, '남산케이블카': 51, '남산타운 아파트': 51, '덕수궁 롯데캐슬': 51, '동대문디자인플라자': 51, '롯데캐슬베네치아': 69, '묵정공원 공영주차장': 51, '서울시청': 51, '서울역(한화역사)': 51, '서울전력관리처': 51, '서울직할': 51, '신세계 본점': 51, '쌍용남산플래티넘 아파트': 51, '약수동공영주차장': 51, '전국은행연합회': 51, '한국관광공사 옆 노상 공영주차장': 51, '퇴계로(신당역)': 19, '동호로(장충체육관)': 19, '장충단로(장충단공원)': 19, '퇴계로(회현역)': 19, '세종대로(서울역)': 19, '서소문로(시청역)': 19, '세종대로(시청역2)': 19, '을지로(을지로3가역)': 19, '칠패로(숭례문)': 19, '남산1호터널': 19, '남산3호터널': 19, '소월로(회현역)': 19, '소파로(숭의여자대학교)': 19, '동작대교': 19, 'LIG서울역리가': 19, '서울역센트럴자이아파트': 19, '서울역한라비발디센트럴아파트': 19, '묵정아파트': 19, '덕수궁롯데캐슬아파트 ': 19, '순화동포스코더샵': 19, '한진해모로': 19, '신당삼성임대': 19, '신당푸르지오': 19, '약수하이츠아파트(임대)': 19, '신당현대': 19, '신당삼성(분양)': 19, '신당약수하이츠': 19, '신당남산타운임대': 19, '신당남산타운(분양)': 19, '래미안신당하이베르': 19, '청구e편한세상(분양)': 19, '청구e편한세상임대': 19, '래미안신당하이베르임대': 19, '파라다이스': 19, '신당KCC스위첸아파트': 19, '인현신성': 19, '삼정아트테라스정동': 19, '중림삼성래미안아파트': 19, '중림삼성사이버빌리지': 19, '충무로진양': 19, '충무로엘크루메트로시티2': 19

In [123]:
std_point = (37.56100278, 126.9996417)
map_osm = folium.Map(location=std_point, zoom_start=14) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

kw = {'opacity': 0.5, 'weight': 2}

####### 노드 #################################
for idx, charging2 in node_charging2_junggu.iterrows() : # 충전소
    location = (charging2['latitude'], charging2['longitude']) # 위도, 경도 튜플

    folium.Circle(
        location=location,
        radius=G.degree[charging2['location']] * 10, # 지름이 degree에 비례하도록 설정
        color='red',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)

for i, traffic2 in node_traffic2_junggu.iterrows() : # 교통량 관측점
    location = (traffic2['위도'], traffic2['경도']) # 위도, 경도 튜플
    
    folium.Circle(
        location=location,
        radius=G.degree[traffic2['지점명칭']] * 3, # 지름이 degree에 비례하도록 설정
        color='blue',
        weight=1,
        fill_opacity=0.4,
        opacity=1,
        fill_color='blue',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)


for idx, apt in node_apt_junggu.iterrows() : # 충전소
    location = (apt['좌표X'], apt['좌표Y']) # 위도, 경도 튜플

    folium.Circle(
        location=location,
        radius=G.degree[apt['k-아파트명']] * 10, # 지름이 degree에 비례하도록 설정
        color='black',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='black',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)


####### edge #################################

for idx in range(len(node_traffic2_junggu)) :
    start = tuple(node_traffic2_junggu[['위도','경도']].iloc[idx])

    for i in range(len(node_charging2_junggu)) :
        end = tuple(node_charging2_junggu[['latitude','longitude']].iloc[i])
        folium.PolyLine(
            locations=[start, end],
            color='green',
            line_cap='round',
            **kw,
        ).add_to(map_osm)



for idx in range(len(node_apt_junggu)) :
    start = tuple(node_apt_junggu[['좌표X','좌표Y']].iloc[idx])

    for i in range(len(node_charging2_junggu)) :
        end = tuple(node_charging2_junggu[['latitude','longitude']].iloc[i])
        folium.PolyLine(
            locations=[start, end],
            color='black',
            line_cap='round',
            **kw,
        ).add_to(map_osm)        
# it takes some time.....
map_osm

In [47]:
tuple(node_traffic2_junggu[['위도','경도']].iloc[0])

(37.565712, 127.020912)

In [48]:
node_charging2_junggu[['latitude','longitude']].iloc[0]

latitude     0.0
longitude    0.0
Name: 0, dtype: float64

In [49]:
node_charging2_junggu[['latitude','longitude']].iloc[1]

latitude     0.0
longitude    0.0
Name: 1, dtype: float64

In [50]:
'''
kw = {'opacity': 0.5, 'weight': 2}

# for idx, row in node_traffic2_junggu.iterrows() :
for idx in range(len(node_traffic2_junggu)) :
    # start = tuple(row[['latitude','longitude']].iloc[0])
    start = tuple(node_traffic2_junggu[['위도','경도']].iloc[idx])

    # for i, r in node_charging2_junggu.iterrows() :
    for i in range(len(node_charging2_junggu)) :
        # end = tuple(r[['latitude','longitude']].iloc[0])
        end = tuple(node_charging2_junggu[['latitude','longitude']].iloc[i])
        # print(start, end)
        folium.PolyLine(
            locations=[start, end],
            color='green',
            line_cap='round',
            **kw,
        ).add_to(map_osm)
# it takes some time.....
map_osm
'''

"\nkw = {'opacity': 0.5, 'weight': 2}\n\n# for idx, row in node_traffic2_junggu.iterrows() :\nfor idx in range(len(node_traffic2_junggu)) :\n    # start = tuple(row[['latitude','longitude']].iloc[0])\n    start = tuple(node_traffic2_junggu[['위도','경도']].iloc[idx])\n\n    # for i, r in node_charging2_junggu.iterrows() :\n    for i in range(len(node_charging2_junggu)) :\n        # end = tuple(r[['latitude','longitude']].iloc[0])\n        end = tuple(node_charging2_junggu[['latitude','longitude']].iloc[i])\n        # print(start, end)\n        folium.PolyLine(\n            locations=[start, end],\n            color='green',\n            line_cap='round',\n            **kw,\n        ).add_to(map_osm)\n# it takes some time.....\nmap_osm\n"

In [51]:
'''
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in incheon_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....
map_osm
'''

"\nkw = {'opacity': 0.5, 'weight': 2}\nfor ix, row in incheon_links.iterrows():\n    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])\n    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])\n    folium.PolyLine(\n        locations=[start, end],\n        color='blue',\n        line_cap='round',\n        **kw,\n    ).add_to(map_osm)\n# it takes some time.....\nmap_osm\n"